# [7] Amazon Fine Food Reviews Analysis


Data Source: https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.<br>

Number of reviews: 568,454<br>
Number of users: 256,059<br>
Number of products: 74,258<br>
Timespan: Oct 1999 - Oct 2012<br>
Number of Attributes/Columns in data: 10 

Attribute Information:

1. Id
2. ProductId - unique identifier for the product
3. UserId - unqiue identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review


#### Objective:
Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

<br>
[Q] How to determine if a review is positive or negative?<br>
<br> 
[Ans] We could use the Score/Rating. A rating of 4 or 5 could be cosnidered a positive review. A review of 1 or 2 could be considered negative. A review of 3 is nuetral and ignored. This is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.




## [7.1] Loading the data

The dataset is available in two forms
1. .csv file
2. SQLite Database

In order to load the data, We have used the SQLITE dataset as it easier to query the data and visualise the data efficiently.
<br> 

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3. If the score id above 3, then the recommendation wil be set to "positive". Otherwise, it will be set to "negative".

In [11]:
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU,Conv1D,MaxPooling1D
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import gc
from sklearn.model_selection import train_test_split
from keras.models import load_model

# using the SQLite Table to read data.
con = sqlite3.connect('./database.sqlite') 



#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 




# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0 #'negative'
    return 1 #'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative

/home/ll/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
filtered_data.shape #looking at the number of attributes and size of the data
#filtered_data.head()

(525814, 10)

#  Exploratory Data Analysis

## [7.1.2] Data Cleaning: Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.  Following is an example:

In [13]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
#display

As can be seen above the same user has multiple reviews of the with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text  and on doing analysis it was found that <br>
<br> 
ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)<br>
<br> 
ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on<br>

It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundancy it was decided to eliminate the rows having same parameters.<br>

The method used for the same was that we first sort the data according to ProductId and then just keep the first similar product review and delelte the others. for eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [14]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [15]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [16]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

<b>Observation:-</b> It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calcualtions

In [17]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)
#display


In [18]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [19]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

## 7.2.3  Text Preprocessing: Stemming, stop-word removal and Lemmatization.

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis and making the prediction model.

Hence in the Preprocessing phase we do the following in the order below:-

1. Begin by removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [20]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features,char_level=True)

In [22]:
final.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...


In [23]:
#allows Tokenizer to create an index of the tokenized unique characters. Eg. a=1, b=2
tokenizer.fit_on_texts(list(final['Text']))

In [25]:
#get back a list of sentences with the sequence of indexes which represent each character
list_tokenized = tokenizer.texts_to_sequences(final['Text'])

In [35]:
max_length = 500

features = np.zeros(shape=(len(list_tokenized),max_length),dtype=int)


for i in range(len(list_tokenized)):
    nb_words = len(list_tokenized[i])
    features[i] = [0]*(max_length -nb_words) + list_tokenized[i][:500]
#X_t = pad_sequences(list_tokenized, maxlen=maxlen)

In [37]:
from sklearn.model_selection import StratifiedShuffleSplit

labels = final.Score.values

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1234)
splitter = sss.split(features, labels)
train_index, validation_index = next(splitter)
test_index = validation_index[:int(len(validation_index)/2)]
validation_index = validation_index[int(len(validation_index)/2):]

X_train, y_train = features[train_index], labels[train_index]
X_test, y_test = features[test_index], labels[test_index]
X_valid, yy_valid = features[validation_index], labels[validation_index]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(X_train.shape), 
      "\nValidation set: \t{}".format(X_valid.shape),
      "\nTest set: \t\t{}".format(X_test.shape))

			Feature Shapes:
Train set: 		(291336, 500) 
Validation set: 	(36418, 500) 
Test set: 		(36417, 500)


In [38]:
print(X_train[1])
print(type(X_train[1]))
print(len(X_train[1]))

[24  1 21  4  7  1 23  2  8 17  1 12  6  7  4 20 20  5  6  9  3  2 12  1
 21  6  3 10  1  3 10  2  1 19  5  9  7  4  6  1 15  8  5 16  1 43  8 13
  7  7  2 11 45  7 22  1  1 24  1  5  8 12  2  8  2 12  1  3 10  2  1 20
 11  4  9  3  1  4  7  1  4  1 18  6 15  3  1 15  5  8  1 16 17  1  9  6
  2 14  2  1 21 10  5  1 21  4  7  1  5 20  2  9  6  9 18  1  4  1  9  2
 21  1  5 15 15  6 14  2 22  1  1 36 10  2  1  7  4  6 12  1  3 10  4  3
  1  3 10  2  1 20 11  4  9  3  1 21  4  7  1 12  2 11  6 23  2  8  2 12
  1  3  5  1 10  2  8  1  6  9  1  4  1 23  2  8 17  1 45  7  6 14 25 45
  1  7  3  4  3  2 22  1  1 24  1  6 16 16  2 12  6  4  3  2 11 17  1 14
  5  9  3  4 14  3  2 12  1 32 16  4 37  5  9  1  3  5  1  7  2  2  1 21
 10  4  3  1 24  1 14  5 13 11 12  1 12  5  1  3  5  1 18  2  3  1 16 17
  1  9  6  2 14  2  1  4  1  9  2 21  1 20 11  4  9  3 22  1  1 27 10  2
 17  1 21  2  8  2  1 23  2  8 17  1 10  2 11 20 15 13 11  1  4  9 12  1
 12  2  3  4  6 11  2 12  1  6  9  1  3  2 11 11  6

In [39]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# truncate and/or pad input sequences
max_review_length = 600 # Padding words
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(291336, 600)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0 24  1 21  4  7  1 23  2  8 17  1 12  6  7  4 20 20  5  6  9
  3  2 12  1 21  6  3 10  1  3 10  2  1 19  5  9  7  4  6  1 15  8  5 16
  1 43  8 13  7  7  2 11 45  7 22  1  1 24  1  5  8 12  2  8  2 12  1  3
 10  2  1 20 11  4  9  3  1  4  7  1  4  1 18  6 15  3  1 15  5  8  1 16
 17  1  9  6  2 14  2  1 21 10  5  1 21  4  7  1  5 20  2  9  6  9 18  1
  4  1  9  2 21  1  5 15 15  6 14  2 22  1  1 36 10  2  1  7  4  6 12  1
  3 10  4  3  1  3 10  2  1 20 11  4  9  3  1 21  4  7  1 12  2 11  6 23
  2  8  2 12  1  3  5  1 10  2  8  1  6  9  1  4  1 23  2  8 17  1 45  7
  6 14 25 45  1  7  3  4  3  2 22  1  1 24  1  6 16 16  2 12  6  4  3  2
 11 17  1 14  5  9  3  4 14  3  2 12 

In [40]:
top_words = 5000
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 600, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/home/ll/anaconda3/lib/python3.5/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
291336/291336 [==============================] - 2454s 8ms/step - loss: 0.4194 - acc: 0.8431
Epoch 2/10
291336/291336 [==============================] - 2451s 8ms/step - loss: 0.3644 - acc: 0.8522
Epoch 3/10
291336/291336 [==============================] - 2453s 8ms/step - loss: 0.2947 - acc: 0.8779
Epoch 4/10
291336/291336 [==============================] - 2454s 8ms/step - loss: 0.2509 - acc: 0.8961
Epoch 5/10
291336/291336 [==============================] - 2454s 8ms/step - loss: 0.2192 - acc: 0.9102
Epoch 6/10
291336/291336 [==============================] - 2454s 8ms/step - loss: 0.1979 - acc: 0.9199
Epoch 7/10
291336/291336 [==============================] - 2454s 8ms/step - loss: 0.1822 - acc: 0.9272
Epoch 8/10
291336/291336 [==============================] - 2455s 8ms/step - loss: 0.1710 - acc: 0.9322
Epoch 9/10
291336/291336 [==============================] - 2455s 8ms/step - loss: 0.1621 - acc: 0.9358
Epoch 10/10
291336/291336 [==============================] - 246

In [45]:
model2 = Sequential()
model2.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model2.add(LSTM(100,return_sequences=True))
model2.add(LSTM(100))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 600, 32)           160000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 600, 100)          53200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 293,701
Trainable params: 293,701
Non-trainable params: 0
_________________________________________________________________
None


In [46]:
model2.fit(X_train, y_train, nb_epoch=10, batch_size=64)
# Final evaluation of the model
scores = model2.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

/home/ll/anaconda3/lib/python3.5/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
291336/291336 [==============================] - 4920s 17ms/step - loss: 0.4258 - acc: 0.8430
Epoch 2/10
291336/291336 [==============================] - 4802s 16ms/step - loss: 0.4031 - acc: 0.8443
Epoch 3/10
291336/291336 [==============================] - 4851s 17ms/step - loss: 0.3212 - acc: 0.8683
Epoch 4/10
291336/291336 [==============================] - 4869s 17ms/step - loss: 0.2713 - acc: 0.8878
Epoch 5/10
291336/291336 [==============================] - 4786s 16ms/step - loss: 0.2294 - acc: 0.9056
Epoch 6/10
291336/291336 [==============================] - 4769s 16ms/step - loss: 0.2009 - acc: 0.9184
Epoch 7/10
291336/291336 [==============================] - 4742s 16ms/step - loss: 0.1828 - acc: 0.9261
Epoch 8/10
291336/291336 [==============================] - 4788s 16ms/step - loss: 0.1713 - acc: 0.9309
Epoch 9/10
291336/291336 [==============================] - 4790s 16ms/step - loss: 0.1614 - acc: 0.9355
Epoch 10/10
291336/291336 [============================

In [55]:
import dill
#dill.dump_session('notebook_env.db')